In [5]:
%tensorflow_version 1.x   #Using tensorflow 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':      #use GPU for fast process
  raise SystemError('GPU device not found')
#print('Found GPU at: {}'.format(device_name))

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x   #Using tensorflow 1.x`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [6]:
print(tf.__version__)

1.15.2


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import requests  
file_url = "http://nlp.stanford.edu/data/glove.6B.zip"          #Download glove here is the link
    
r = requests.get(file_url, stream = True)  
  
with open("glove.6B.zip", "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
         if block:  
             file.write(block) 

In [9]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.


In [10]:
cd '/content/go'      #downloaded in server but not in my drive it is fast to access

[Errno 2] No such file or directory: '/content/go #downloaded in server but not in my drive it is fast to access'
/content


In [11]:
!unzip  '/content/glove.6B.zip' -d go

Archive:  /content/glove.6B.zip
replace go/glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: go/glove.6B.50d.txt     
replace go/glove.6B.100d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: go/glove.6B.100d.txt    y

replace go/glove.6B.200d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: go/glove.6B.200d.txt    y

replace go/glove.6B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: go/glove.6B.300d.txt    y
y



In [12]:
'''import numpy as np
np.random.seed(42)
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import keras
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D,GlobalAveragePooling1D
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
import warnings
warnings.filterwarnings('ignore')'''
import numpy as np
import pandas as pd
import re
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [257]:
EMBEDDING_FILE = '/content/go/glove.6B.300d.txt'      
train = pd.read_csv('/content/drive/My Drive/Hate Speech/english_dataset.tsv', sep = '\t', encoding="latin-1")
test = pd.read_csv('/content/drive/My Drive/Hate Speech/hasoc2019_en_test-2919.tsv', sep = '\t', encoding="latin-1")

In [258]:
train=train[train.task_3!="NONE"]       #drop none
test=test[test.task_3!="NONE"]
train=train.reset_index(drop=True)
test=test.reset_index(drop=True)
X_train = train["text"]
y_train = train["task_3"]                     #For task 3         
X_test = test["text"]
y_test = test["task_3"]

In [259]:
y_train

0       TIN
1       TIN
2       TIN
3       TIN
4       TIN
       ... 
2256    UNT
2257    UNT
2258    TIN
2259    UNT
2260    UNT
Name: task_3, Length: 2261, dtype: object

In [260]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [261]:
#Remove stopwords from the text
stop_words = set(stopwords.words('english')) 
ix=0
for i in X_train:
  word_tokens = (i[0].split())  
  for w in word_tokens: 
    if w not in stop_words: 
        X_train[ix]=X_train[ix]+(w) 
  ix=ix+1
ix=0
for i in X_test:
  word_tokens = (i[0].split())  
  for w in word_tokens: 
    if w not in stop_words: 
        X_test[ix]= X_test[ix]+(w) 
  ix=ix+1

In [262]:
#remove punctuations
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, '')
    return x

In [263]:
ix=0
for i in X_train:
  X_train[ix]=clean_text(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=clean_text(i)
  ix=ix+1

In [264]:
#clean the numbers
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [265]:
ix=0
for i in X_train:
  X_train[ix]=clean_numbers(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=clean_numbers(i)
  ix=ix+1

In [266]:
#remove contractions
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)


In [267]:
ix=0
for i in X_train:
  X_train[ix]=replace_contractions(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=replace_contractions(i)
  ix=ix+1

In [268]:
#Stemming
'''Stemming is the process of converting words to their base forms using crude Heuristic rules.
   For example, one rule could be to remove ’s’ from the end of any word, so that ‘cats’ becomes ‘cat’. 
   or another rule could be to replace ‘ies’ with ‘i’ so that ‘ponies becomes ‘poni’.'''
from nltk.stem import  SnowballStemmer
from nltk.tokenize.toktok import ToktokTokenizer
def stem_text(text):
    tokenizer = ToktokTokenizer()
    stemmer = SnowballStemmer('english')
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

In [269]:
ix=0
for i in X_train:
  X_train[ix]=stem_text(i)
  ix=ix+1
ix=0
for i in X_test:
  X_test[ix]=stem_text(i)
  ix=ix+1

In [270]:
type(X_train)

pandas.core.series.Series

In [271]:
#After removing or cleaning the text
max_features = 11118  #No. of different words in total text
maxlen = 70           #Padding length
embed_size = 300      #No. of dimensions of glove

threshold = 0.35

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [272]:
y_train = y_train.map({'TIN':0, 'UNT': 1})    #Map accordingly 
y_test = y_test.map({'TIN':0, 'UNT': 1})

In [273]:
y_train=keras.utils.to_categorical(y_train, num_classes=2)  #one hot encoding
y_test=keras.utils.to_categorical(y_test, num_classes=2)


In [274]:
y=[]
for i in y_train:
 y.append(np.argmax(i))

In [275]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [276]:
num_filters = 42
filter_sizes = [1,2,3,5]  #ngrams

In [301]:
#CNN

def get_model():    
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
#    x = SpatialDropout1D(0.4)(x)
    x = Reshape((maxlen, embed_size, 1))(x)
    
    conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embed_size),
                                 kernel_initializer='he_normal', activation='tanh')(x)
    conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embed_size),
                                 kernel_initializer='he_normal', activation='tanh')(x)
    conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embed_size), 
                                 kernel_initializer='he_normal', activation='tanh')(x)
    conv_3 = Conv2D(num_filters, kernel_size=(filter_sizes[3], embed_size),
                                 kernel_initializer='he_normal', activation='tanh')(x)
    
    maxpool_0 = MaxPool2D(pool_size=(maxlen - filter_sizes[0] + 1, 1))(conv_0)
    maxpool_1 = MaxPool2D(pool_size=(maxlen - filter_sizes[1] + 1, 1))(conv_1)
    maxpool_2 = MaxPool2D(pool_size=(maxlen - filter_sizes[2] + 1, 1))(conv_2)
    maxpool_3 = MaxPool2D(pool_size=(maxlen - filter_sizes[3] + 1, 1))(conv_3)
        
    z = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3])   
    z = Flatten()(z)
    z = Dropout(0.1)(z)
        
    outp = Dense(2, activation="sigmoid")(z)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

model = get_model()

In [277]:
#model of bi-lstm
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D,concatenate
def model_lstm_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
        64*70(maxlen)*2(bidirection concat)
    CuDNNLSTM is fast implementation of LSTM layer in Keras which only runs on GPU
    '''
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(2, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [278]:
#model of lstm 
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D,concatenate
def model_slstm_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
        64*70(maxlen)*2(bidirection concat)
    LSTM is fast implementation of LSTM layer in Keras which only runs on GPU
    '''
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(2, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [279]:
#model of gru 
from keras.layers import Dense, Input, GRU, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D,concatenate
def model_gru_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
        64*70(maxlen)*2(bidirection concat)
    GRU is fast implementation of LSTM layer in Keras which only runs on GPU
    '''
    x = Bidirectional(GRU(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(2, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [280]:
#model of Simple RNN 
from keras.layers import Dense, Input, SimpleRNN, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D,concatenate
def model_rnn_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
        64*70(maxlen)*2(bidirection concat)
    SimpleRNN is fast implementation of LSTM layer in Keras which only runs on GPU
    '''
    x = Bidirectional(SimpleRNN(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(2, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [281]:
print(tf.__version__)

1.15.2


In [302]:
hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 3s - loss: 0.4333 - accuracy: 0.8314 - val_loss: 0.3889 - val_accuracy: 0.8772
Epoch 2/10
 - 0s - loss: 0.3254 - accuracy: 0.9022 - val_loss: 0.3476 - val_accuracy: 0.8860
Epoch 3/10
 - 0s - loss: 0.2751 - accuracy: 0.9031 - val_loss: 0.3204 - val_accuracy: 0.8860
Epoch 4/10
 - 0s - loss: 0.2570 - accuracy: 0.9036 - val_loss: 0.3166 - val_accuracy: 0.8860
Epoch 5/10
 - 0s - loss: 0.2316 - accuracy: 0.9036 - val_loss: 0.3163 - val_accuracy: 0.8860
Epoch 6/10
 - 0s - loss: 0.2099 - accuracy: 0.9041 - val_loss: 0.3012 - val_accuracy: 0.8947
Epoch 7/10
 - 0s - loss: 0.1931 - accuracy: 0.9041 - val_loss: 0.2994 - val_accuracy: 0.8947
Epoch 8/10
 - 0s - loss: 0.1760 - accuracy: 0.9045 - val_loss: 0.2983 - val_accuracy: 0.8947
Epoch 9/10
 - 0s - loss: 0.1602 - accuracy: 0.9041 - val_loss: 0.2873 - val_accuracy: 0.8947
Epoch 10/10
 - 0s - loss: 0.1462 - accuracy: 0.9050 - val_loss: 0.2951 - val_accuracy: 0.8947


In [282]:
ls_model = model_lstm_atten(embedding_matrix) #model of bi-lstm with attention

In [283]:
sls_model = model_slstm_atten(embedding_matrix) #model of lstm with attention

In [284]:
gru_model = model_gru_atten(embedding_matrix) #model of gru with attention

In [285]:
rnn_model = model_rnn_atten(embedding_matrix) #model of rnn with attention

In [286]:
batch_size = 256
epochs = 10

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95,
                                              random_state=42) #split this into train and validation

In [287]:
ls_hist = ls_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)      #Train bi-lstm with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 2s - loss: 0.5023 - accuracy: 0.8938 - val_loss: 0.3440 - val_accuracy: 0.8947
Epoch 2/10
 - 0s - loss: 0.3531 - accuracy: 0.9031 - val_loss: 0.3367 - val_accuracy: 0.8947
Epoch 3/10
 - 0s - loss: 0.3261 - accuracy: 0.9031 - val_loss: 0.3349 - val_accuracy: 0.8947
Epoch 4/10
 - 0s - loss: 0.3212 - accuracy: 0.9031 - val_loss: 0.3249 - val_accuracy: 0.8947
Epoch 5/10
 - 0s - loss: 0.3173 - accuracy: 0.9031 - val_loss: 0.3226 - val_accuracy: 0.8947
Epoch 6/10
 - 0s - loss: 0.3019 - accuracy: 0.9031 - val_loss: 0.3174 - val_accuracy: 0.8947
Epoch 7/10
 - 0s - loss: 0.2903 - accuracy: 0.9031 - val_loss: 0.3114 - val_accuracy: 0.8947
Epoch 8/10
 - 0s - loss: 0.2704 - accuracy: 0.9031 - val_loss: 0.3024 - val_accuracy: 0.8947
Epoch 9/10
 - 0s - loss: 0.2445 - accuracy: 0.9031 - val_loss: 0.2909 - val_accuracy: 0.8947
Epoch 10/10
 - 0s - loss: 0.2034 - accuracy: 0.9031 - val_loss: 0.2831 - val_accuracy: 0.8947


In [288]:
sls_hist = sls_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)    #Train lstm with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 4s - loss: 0.4753 - accuracy: 0.9036 - val_loss: 0.3386 - val_accuracy: 0.8947
Epoch 2/10
 - 1s - loss: 0.3467 - accuracy: 0.9031 - val_loss: 0.3322 - val_accuracy: 0.8947
Epoch 3/10
 - 1s - loss: 0.3216 - accuracy: 0.9031 - val_loss: 0.3317 - val_accuracy: 0.8947
Epoch 4/10
 - 1s - loss: 0.3120 - accuracy: 0.9031 - val_loss: 0.3264 - val_accuracy: 0.8947
Epoch 5/10
 - 1s - loss: 0.3085 - accuracy: 0.9031 - val_loss: 0.3186 - val_accuracy: 0.8947
Epoch 6/10
 - 1s - loss: 0.2946 - accuracy: 0.9031 - val_loss: 0.3115 - val_accuracy: 0.8947
Epoch 7/10
 - 1s - loss: 0.2734 - accuracy: 0.9031 - val_loss: 0.3010 - val_accuracy: 0.8947
Epoch 8/10
 - 1s - loss: 0.2502 - accuracy: 0.9031 - val_loss: 0.2835 - val_accuracy: 0.8947
Epoch 9/10
 - 1s - loss: 0.2143 - accuracy: 0.9031 - val_loss: 0.2564 - val_accuracy: 0.8947
Epoch 10/10
 - 1s - loss: 0.1750 - accuracy: 0.9031 - val_loss: 0.2208 - val_accuracy: 0.8947


In [289]:
gru_hist = gru_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)      #Train GRU with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 4s - loss: 0.5801 - accuracy: 0.7145 - val_loss: 0.3734 - val_accuracy: 0.8947
Epoch 2/10
 - 1s - loss: 0.3398 - accuracy: 0.9031 - val_loss: 0.3477 - val_accuracy: 0.8947
Epoch 3/10
 - 2s - loss: 0.3464 - accuracy: 0.9031 - val_loss: 0.3353 - val_accuracy: 0.8947
Epoch 4/10
 - 2s - loss: 0.3187 - accuracy: 0.9031 - val_loss: 0.3315 - val_accuracy: 0.8947
Epoch 5/10
 - 2s - loss: 0.3113 - accuracy: 0.9031 - val_loss: 0.3239 - val_accuracy: 0.8947
Epoch 6/10
 - 1s - loss: 0.3004 - accuracy: 0.9031 - val_loss: 0.3186 - val_accuracy: 0.8947
Epoch 7/10
 - 2s - loss: 0.2827 - accuracy: 0.9031 - val_loss: 0.3115 - val_accuracy: 0.8947
Epoch 8/10
 - 1s - loss: 0.2647 - accuracy: 0.9031 - val_loss: 0.3060 - val_accuracy: 0.8947
Epoch 9/10
 - 1s - loss: 0.2370 - accuracy: 0.9031 - val_loss: 0.2904 - val_accuracy: 0.8947
Epoch 10/10
 - 1s - loss: 0.2073 - accuracy: 0.9031 - val_loss: 0.2822 - val_accuracy: 0.8947


In [290]:
rnn_hist = rnn_model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs,
                 validation_data=(X_val, y_val),
                  verbose=2)      #Train RNN with attention

Train on 2147 samples, validate on 114 samples
Epoch 1/10
 - 3s - loss: 0.3604 - accuracy: 0.9031 - val_loss: 0.3318 - val_accuracy: 0.8947
Epoch 2/10
 - 1s - loss: 0.3139 - accuracy: 0.9031 - val_loss: 0.3251 - val_accuracy: 0.8947
Epoch 3/10
 - 1s - loss: 0.3049 - accuracy: 0.9031 - val_loss: 0.3236 - val_accuracy: 0.8947
Epoch 4/10
 - 1s - loss: 0.2916 - accuracy: 0.9031 - val_loss: 0.3195 - val_accuracy: 0.8947
Epoch 5/10
 - 1s - loss: 0.2714 - accuracy: 0.9031 - val_loss: 0.3123 - val_accuracy: 0.8947
Epoch 6/10
 - 1s - loss: 0.2478 - accuracy: 0.9031 - val_loss: 0.3397 - val_accuracy: 0.8947
Epoch 7/10
 - 1s - loss: 0.2317 - accuracy: 0.9031 - val_loss: 0.3026 - val_accuracy: 0.8947
Epoch 8/10
 - 1s - loss: 0.1906 - accuracy: 0.9031 - val_loss: 0.3529 - val_accuracy: 0.8947
Epoch 9/10
 - 1s - loss: 0.1540 - accuracy: 0.9078 - val_loss: 0.2627 - val_accuracy: 0.8947
Epoch 10/10
 - 1s - loss: 0.1072 - accuracy: 0.9711 - val_loss: 0.3622 - val_accuracy: 0.9123


In [291]:
from sklearn.metrics import classification_report

In [292]:
y_t=[]
for i in y_test:
  y_t.append(np.argmax(i))    #Using argmax to find the index of one hot encoding

In [303]:
y__pred =model.predict(x_test)      #predict y_test by bi-lstm with attention
y__p=[]
for i in y__pred:
  y__p.append(np.argmax(i))

In [304]:
print('Model = CNN\n')  
print(classification_report(y_t, y__p))

Model = CNN

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       245
           1       0.00      0.00      0.00        43

    accuracy                           0.85       288
   macro avg       0.43      0.50      0.46       288
weighted avg       0.72      0.85      0.78       288



In [293]:
y_ls_pred = ls_model.predict(x_test)      #predict y_test by bi-lstm with attention
y_ls_p=[]
for i in y_ls_pred:
  y_ls_p.append(np.argmax(i))


In [294]:
print('Model = Attention Bidirectonal LSTM\n')  
print(classification_report(y_t, y_ls_p))

Model = Attention Bidirectonal LSTM

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       245
           1       0.00      0.00      0.00        43

    accuracy                           0.85       288
   macro avg       0.43      0.50      0.46       288
weighted avg       0.72      0.85      0.78       288



In [295]:
y_sls_pred = sls_model.predict(x_test)  #predict y_test by lstm with attention
y_sls_p=[]
for i in y_sls_pred:
  y_sls_p.append(np.argmax(i))

In [296]:
print('Model = Attention LSTM\n')
print(classification_report(y_t, y_sls_p))

Model = Attention LSTM

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       245
           1       0.00      0.00      0.00        43

    accuracy                           0.85       288
   macro avg       0.43      0.50      0.46       288
weighted avg       0.72      0.85      0.78       288



In [297]:
y_gru_pred = gru_model.predict(x_test)    #predict y_test by GRU with attention
y_gru_p=[]
for i in y_gru_pred:
  y_gru_p.append(np.argmax(i))

In [298]:
print('Model = Attention GRU\n')
print(classification_report(y_t, y_gru_p))

Model = Attention GRU

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       245
           1       0.00      0.00      0.00        43

    accuracy                           0.85       288
   macro avg       0.43      0.50      0.46       288
weighted avg       0.72      0.85      0.78       288



In [299]:
y_rnn_pred = rnn_model.predict(x_test)    #predict y_test by RNN with attention
y_rnn_p=[]
for i in y_rnn_pred:
  y_rnn_p.append(np.argmax(i))

In [300]:
print('Model = Attention SimpleRNN\n')
print(classification_report(y_t, y_rnn_p))

Model = Attention SimpleRNN

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       245
           1       0.00      0.00      0.00        43

    accuracy                           0.85       288
   macro avg       0.43      0.50      0.46       288
weighted avg       0.72      0.85      0.78       288

